In [4]:
import numpy as np
from scipy.linalg import lstsq
from scipy.linalg import null_space
from scipy.integrate import quad
import pandas as pd
import openpyxl  # 使用pd.read_excel中需要保证openpyxl库已安装，但可以不导入。


In [5]:
wf = pd.read_excel(
    r'D:\研究死\电子结构与理论计算\补充材料\ncpp\wavefunc.xlsx', 
    sheet_name="Sheet1",  # 选择特定工作表
    nrows=1780,             # 只读取 10 行
    usecols="A:F"         # 读取 A 到 D 列
)
print(wf)

              r            3P        3S            2P        2S        1S
0      0.000000  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000
1      0.000065  8.533000e-08  0.000502  3.660390e-07  0.001746  0.006586
2      0.000066  8.670600e-08  0.000506  3.719410e-07  0.001760  0.006639
3      0.000066  8.810400e-08  0.000510  3.779390e-07  0.001774  0.006692
4      0.000067  8.952500e-08  0.000514  3.840330e-07  0.001788  0.006746
...         ...           ...       ...           ...       ...       ...
1775  94.911290  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000
1776  95.673626  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000
1777  96.442085  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000
1778  97.216716  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000
1779  97.997569  0.000000e+00  0.000000  0.000000e+00  0.000000  0.000000

[1780 rows x 6 columns]


In [6]:

#r_0 = pd.to_numeric(wf['r'], errors='coerce').values
r_0=wf['r'].values
P_3=wf['3P'].values
P_2=wf['2P'].values
S_3=wf['3S'].values
S_2=wf['2S'].values
S_1=wf['1S'].values
print(r_0)




[0.00000000e+00 6.51344260e-05 6.56575910e-05 ... 9.64420848e+01
 9.72167158e+01 9.79975688e+01]


In [7]:
#小写p(r)系列构建关于c0, c2, c3, c4的线性方程组
#range(i, j) produces i, i+1, i+2, ..., j-1. start defaults to 0
def p_r(r):
    A=np.zeros(4)
    A[0]=1
    for i in range(1,len(A)):
        A[i]=r**(i+1)
    return A

def dp_r(r):
    dA=np.zeros(4)
    for i in range(1,4):
        dA[i]=(i+1)*r**(i)
    return dA

def ddp_r(r):
    ddA=np.zeros(4)
    for i in range(1,4):
        ddA[i]=i*(i+1)*r**(i-1)
    return ddA
# A[0]~c_0, A[1]~c_2, A[2]~c_3, A[3]~c_4

'''p_3p=np.array([p_r(2.14),dp_r(2.14),ddp_r(2.14)])
print(p_3p)
print(null_space(p_3p))'''

#l_vec(r)写出关于线性方程组的矩阵
def l_vec(r):
    return np.array([p_r(r), dp_r(r), ddp_r(r)])

print(l_vec(2))
print(null_space(l_vec(2.14)))
A=np.zeros(4)
print(len(A))

[[ 1.  4.  8. 16.]
 [ 0.  4. 12. 32.]
 [ 0.  2. 12. 48.]]
[[-0.54206174]
 [ 0.71018657]
 [-0.44248385]
 [ 0.07753806]]
4


In [8]:
def coeff_generation(r, orbital):
    closest_index = np.argmin(np.abs(r_0 - r))  # 找到最接近值的索引
    start_index = closest_index-5
    end_index = closest_index+6
    r_new = r_0[start_index:end_index]
    orbital_new = orbital[start_index:end_index]
    coefficients = np.polyfit(r_new, orbital_new, 3)
    return coefficients
print(coeff_generation(2.14, P_3))
#大写P是rR_AE的代指，此处已经经过在截断半径处的局部三次多项式拟合
def P(r, coeff):
    return sum(coeff[3-i] * r**i for i in range(0, len(coeff)))

def dP(r, coeff):
    return sum(i * coeff[3-i] * r**(i-1) for i in range(0, len(coeff)))

def ddP(r, coeff):  
    return sum(i * (i-1) * coeff[3-i] * r**(i-2) for i in range(0, len(coeff)))

def r_vec(r,orbital,l):
    coeff=coeff_generation(r,orbital)

    de0=np.log(abs(P(r,coeff)/r**(l+1)))
    de1=dP(r,coeff)/P(r,coeff)-(l+1)/r
    de2=ddP(r,coeff)/P(r,coeff)-\
    (dP(r,coeff)/P(r,coeff))**2+(l+1)/r**2
    return np.array([de0,de1,de2])


[-0.08737399  0.73568108 -1.95035797  1.04065984]


In [9]:
r_vec_3p=r_vec(2.14,P_3,1)
l_vec_3p=l_vec(2.14)
print(r_vec_3p)
print(l_vec_3p)

x, residuals, rank, s = lstsq(l_vec_3p,r_vec_3p)

print("广义解:", x)
print("矩阵的秩:", rank)
print("奇异值:", s)

[-1.99920184 -0.93126352 -0.12671804]
[[ 1.          4.5796      9.800344   20.97273616]
 [ 0.          4.28       13.7388     39.201376  ]
 [ 0.          2.         12.84       54.9552    ]]
广义解: [-0.69651488 -0.46457527  0.1058426  -0.010128  ]
矩阵的秩: 3
奇异值: [73.81423413  6.4451246   0.74489052]


In [10]:
def I_right(r,orbital):
    closest_index=  np.argmin(np.abs(r_0 - r))
    end_index = closest_index
    r_new = r_0[:end_index]
    orbital_new = orbital[:end_index]
    o_new=orbital_new**2
    I_right=np.trapz(o_new,r_new)
    return I_right

I_r_3p=I_right(2.14,P_3)
print(I_r_3p)

0.32903296751654143


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)


In [11]:
def I_left(x,null_space,c,l,r_c):
    def integrand(r):
        exponent=x[0]+null_space[0]*c+(x[1]+null_space[1]*c)*r**2+\
        (x[2]+null_space[2]*c)*r**3+(x[3]+null_space[3]*c)*r**4
        return r**(2*l+2)*np.exp(exponent)
    return quad(integrand,0,r_c)[0]

# 定义误差函数
def F(I_right, x, null_space, c, l, r_c):
    return I_left(x,null_space,c,l,r_c) - I_right

print(F(0.3,x,null_space(l_vec(2.14)),100,1,2.14))

0.47849873201181586


In [12]:
r_vec_1s=r_vec(0.15,S_1,0)
l_vec_1s=l_vec(0.15)
print(r_vec_1s)
print(l_vec_1s)

x_1s, residuals, rank, s = lstsq(l_vec_1s,r_vec_1s)

print("广义解:", x_1s)
print(null_space(l_vec_1s))

I_r_1s=I_right(0.15,S_1)
print(F(I_r_1s,x_1s,null_space(l_vec(0.15)),-50000,0,0.15))

[  2.56739001 -13.3602284    3.22262921]
[[1.0000e+00 2.2500e-02 3.3750e-03 5.0625e-04]
 [0.0000e+00 3.0000e-01 6.7500e-02 1.3500e-02]
 [0.0000e+00 2.0000e+00 9.0000e-01 2.7000e-01]]
广义解: [  3.90299167 -87.34455989 175.44676067  74.10986809]
[[-1.56543861e-04]
 [ 4.17450295e-02]
 [-3.71066929e-01]
 [ 9.27667323e-01]]
0.2505100443405578


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)


In [13]:
r_vec_2p=r_vec(0.39,P_2,1)
l_vec_2p=l_vec(0.39)
print(r_vec_2p)
print(l_vec_2p)

x_2p, residuals, rank, s = lstsq(l_vec_2p,r_vec_2p)


I_r_2p=I_right(0.39,P_2)
print(F(I_r_2p,x_2p,null_space(l_vec(0.39)),-2000,1,0.46))

[ 2.15148296 -5.15298794  2.87403202]
[[1.         0.1521     0.059319   0.02313441]
 [0.         0.78       0.4563     0.237276  ]
 [0.         2.         2.34       1.8252    ]]
0.3209503923523762


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)


In [14]:
def cal(x, null_space,c):
    for i in range(len(x)):
        x[i]=x[i]+null_space[i]*c
    return x



In [15]:
def false_position_method(orbital, c_low, c_high, l, r_c, tol=1e-4, max_iter=500):
    i=0
    I_right1=I_right(r_c,orbital)
    l_vec1=l_vec(r_c)
    r_vec1=r_vec(r_c,orbital,l)
    x, residuals, rank, s = lstsq(l_vec1,r_vec1)
    null_space1=null_space(l_vec1)
    for i in range(max_iter):
        # 计算两端误差值
        f_low = F(I_right1, x, null_space1, c_low, l, r_c)
        f_high = F(I_right1, x, null_space1, c_high, l, r_c)

        # 更新新点
        c_new = c_low - f_low * (c_high - c_low) / (f_high - f_low)
        f_new = F(I_right1, x, null_space1, c_new, l, r_c)

        # 检查收敛
        if abs(f_new) < tol:
            return cal(x, null_space1, c_new)

        # 更新区间
        if f_new * f_low < 0:
            c_high = c_new
            i = i+1
        else:
            c_low = c_new
            i = i+1

    raise ValueError("未能收敛")

In [17]:
c_3p=false_position_method(P_3, -0.5, 0.5, 1, 2.14)
print(c_3p)



C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)


[-960.98371524 1257.66347133 -783.77456026  137.35209212]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2359091353.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c


In [18]:
c_1s=false_position_method(S_1, -50000, -40000, 0, 0.15,1e-4,1000)
print(c_1s)

[ 1.13469059e+01 -2.07238834e+03  1.78202804e+04 -4.40379742e+04]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)
C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2359091353.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c


In [19]:
c_2s=false_position_method(S_2, -500, -400, 0, 0.46,1e-4,10000)
print(c_2s)

[   5.77534861 -122.44684585  350.39037189 -287.61894479]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)
C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2359091353.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c


In [20]:
c_2p=false_position_method(P_2, -2000, 00, 1, 0.39,1e-4,1000)
print(c_2p)

[   12.93358024  -384.25389636  1277.35314927 -1215.00358566]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)
C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2359091353.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c


In [21]:
c_3s=false_position_method(S_3, 0, 500, 0, 1.78,1e-4,1000)
print(c_3s)

C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2655798078.py:7: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  I_right=np.trapz(o_new,r_new)


[-42.03067905  78.48269481 -58.82015157  12.3735376 ]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_33876\2359091353.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c
